# PN Junction Diode

## Understanding the Fundamental Semiconductor Device

The PN junction diode is the most fundamental semiconductor device and forms the basis for understanding more complex devices like transistors and solar cells.

**Learning Objectives:**
- Understand PN junction physics
- Simulate equilibrium band diagrams
- Analyze forward and reverse bias characteristics
- Extract diode parameters from I-V curves

---

## 1. PN Junction Physics

### 1.1 Structure

A PN junction consists of:
- **P-region**: Semiconductor doped with acceptors (holes are majority carriers)
- **N-region**: Semiconductor doped with donors (electrons are majority carriers)
- **Depletion region**: The space charge region at the junction

```
    P-region          |    N-region
  (Acceptors Na)      |  (Donors Nd)
                      |
  Holes (majority)    |  Electrons (majority)
  Electrons (minority)|  Holes (minority)
                      |
         <-- Depletion Region -->
```

### 1.2 Key Parameters

- **Built-in potential** ($V_{bi}$): The potential barrier at equilibrium
  $$V_{bi} = \frac{kT}{q} \ln\left(\frac{N_a N_d}{n_i^2}\right)$$

- **Depletion width** ($W$): Width of the space charge region
  $$W = \sqrt{\frac{2\epsilon_s}{q}\left(\frac{1}{N_a} + \frac{1}{N_d}\right)(V_{bi} - V)}$$

- **Diode current** (ideal):
  $$I = I_s\left(e^{qV/kT} - 1\right)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_pn_diode, Solve

# Physical constants
q = 1.6e-19      # Elementary charge (C)
kT = 0.0259      # Thermal voltage at 300K (eV)
ni = 1.5e10      # Intrinsic carrier concentration for Si (cm^-3)
eps_si = 11.7 * 8.85e-14  # Silicon permittivity (F/cm)

# Calculate built-in potential
Na = 1e17  # Acceptor concentration (cm^-3)
Nd = 1e17  # Donor concentration (cm^-3)

Vbi = kT * np.log(Na * Nd / ni**2)
print(f"Built-in potential Vbi = {Vbi:.3f} V")

# Calculate depletion width at equilibrium
W = np.sqrt(2 * eps_si / q * (1/Na + 1/Nd) * Vbi) * 1e4  # Convert to microns
print(f"Depletion width W = {W:.4f} um")

---

## 2. Creating a PN Diode Simulation

Let's create a basic PN diode and examine its equilibrium properties.

In [ ]:
# Create a PN diode with symmetric doping
sim_eq = create_pn_diode(
    # Geometry
    length=2.0,              # 2 um total length
    width=1.0,               # 1 um width
    junction_position=0.5,   # Junction at center
    
    # Mesh
    nx=200,                  # Fine mesh for accurate results
    ny=3,                    # Minimal y-mesh (quasi-1D)
    
    # Doping
    p_doping=1e17,           # P-region: 1e17 cm^-3
    n_doping=1e17,           # N-region: 1e17 cm^-3
    
    # Models
    temperature=300,         # Room temperature
    srh=True,               # SRH recombination
    conmob=True,            # Concentration-dependent mobility
    
    # Output
    log_bands_eq=True        # Log band diagram at equilibrium
)

print("PN Diode Simulation Created")
print("="*40)
print(f"Device length: 2.0 um")
print(f"P-doping: 1e17 cm^-3")
print(f"N-doping: 1e17 cm^-3")
print(f"Junction position: 1.0 um (center)")

In [ ]:
# View the generated PADRE input
print("Generated PADRE Input Deck:")
print("="*60)
print(sim_eq.generate_deck())

---

## 3. Equilibrium Band Diagram

At equilibrium (zero bias), the Fermi level is constant throughout the device. The band bending creates the built-in potential barrier.

In [ ]:
# Create simulation for band diagram analysis
sim_bands = create_pn_diode(
    length=2.0,
    p_doping=1e17,
    n_doping=1e17,
    log_bands_eq=True
)

# The simulation will output:
# - cbeq: Conduction band edge (Ec)
# - vbeq: Valence band edge (Ev)

print("Band diagram outputs configured:")
print("  - cbeq: Ec (conduction band)")
print("  - vbeq: Ev (valence band)")
print("\nTo run: result = sim_bands.run()")

### Understanding the Band Diagram

The equilibrium band diagram shows:

1. **P-region (left)**: 
   - Fermi level near valence band
   - Bands curved upward toward junction

2. **N-region (right)**:
   - Fermi level near conduction band
   - Bands curved downward toward junction

3. **Junction**:
   - Band bending = built-in potential
   - Depletion region where bands curve

In [ ]:
# Theoretical band diagram (for visualization)
x = np.linspace(0, 2, 500)
junction = 1.0

# Simplified band model
Eg = 1.12  # Silicon bandgap (eV)
chi = 4.05  # Electron affinity (eV)

# Band edges (simplified - actual simulation is more accurate)
Ef = 0  # Reference

# P-side: Ef near Ev
# N-side: Ef near Ec
# Transition at junction

def band_profile(x, Vbi, junction, W):
    """Simplified band profile for visualization"""
    Ec = np.zeros_like(x)
    Ev = np.zeros_like(x)
    
    for i, xi in enumerate(x):
        if xi < junction - W/2:
            # P-region (flat)
            Ec[i] = Vbi + Eg/2
            Ev[i] = Vbi - Eg/2
        elif xi > junction + W/2:
            # N-region (flat)
            Ec[i] = Eg/2
            Ev[i] = -Eg/2
        else:
            # Depletion region (linear approximation)
            frac = (xi - (junction - W/2)) / W
            V_drop = Vbi * (1 - frac)
            Ec[i] = V_drop + Eg/2
            Ev[i] = V_drop - Eg/2
    return Ec, Ev

Ec, Ev = band_profile(x, Vbi, junction, W)

plt.figure(figsize=(10, 6))
plt.plot(x, Ec, 'b-', linewidth=2, label='Ec (Conduction Band)')
plt.plot(x, Ev, 'r-', linewidth=2, label='Ev (Valence Band)')
plt.axhline(y=0, color='g', linestyle='--', linewidth=1.5, label='Ef (Fermi Level)')
plt.axvline(x=junction, color='gray', linestyle=':', alpha=0.5, label='Junction')

# Mark depletion region
plt.axvspan(junction - W/2, junction + W/2, alpha=0.2, color='yellow', label='Depletion Region')

plt.xlabel('Position (um)', fontsize=12)
plt.ylabel('Energy (eV)', fontsize=12)
plt.title('PN Junction Equilibrium Band Diagram (Theoretical)', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(0, 2)

# Add annotations
plt.annotate('P-region', xy=(0.3, 0.3), fontsize=12)
plt.annotate('N-region', xy=(1.5, 0.3), fontsize=12)
plt.annotate(f'Vbi = {Vbi:.2f}V', xy=(1.0, Vbi/2), fontsize=10,
             arrowprops=dict(arrowstyle='->', color='black'),
             xytext=(1.3, 0.6))

plt.tight_layout()
plt.show()

---

## 4. Forward Bias Characteristics

Under forward bias (positive voltage on P-side):
- Barrier height decreases
- Current increases exponentially
- Minority carrier injection increases

In [ ]:
# Create diode with forward bias sweep
sim_forward = create_pn_diode(
    length=2.0,
    p_doping=1e17,
    n_doping=1e17,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="forward_iv",
    
    # Forward bias sweep: 0 to 0.8V in 0.05V steps
    forward_sweep=(0.0, 0.8, 0.05),
    
    # Also log band diagrams during sweep
    log_bands_eq=True,
    log_bands_bias=True
)

print("Forward Bias Simulation Configured")
print("="*40)
print("Voltage sweep: 0V to 0.8V")
print("Step size: 0.05V")
print("Number of bias points: 16")

In [ ]:
# Theoretical I-V characteristic
V = np.linspace(0, 0.8, 100)
Is = 1e-14  # Saturation current (A) - typical value
I_ideal = Is * (np.exp(V / kT) - 1)

plt.figure(figsize=(12, 5))

# Linear scale
plt.subplot(1, 2, 1)
plt.plot(V, I_ideal * 1e6, 'b-', linewidth=2)
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (uA)', fontsize=12)
plt.title('Forward I-V (Linear Scale)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)

# Semi-log scale
plt.subplot(1, 2, 2)
plt.semilogy(V, np.abs(I_ideal), 'b-', linewidth=2)
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Forward I-V (Semi-log Scale)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.8)
plt.ylim(1e-14, 1e-3)

# Add slope annotation
plt.annotate('Slope = q/kT\n(60 mV/decade)', xy=(0.4, 1e-8), fontsize=10)

plt.tight_layout()
plt.show()

---

## 5. Reverse Bias Characteristics

Under reverse bias (negative voltage on P-side):
- Barrier height increases
- Depletion region widens
- Only small reverse saturation current flows

In [ ]:
# Create diode with reverse bias sweep
sim_reverse = create_pn_diode(
    length=2.0,
    p_doping=1e17,
    n_doping=1e17,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="reverse_iv",
    
    # Reverse bias sweep: 0 to -5V
    reverse_sweep=(0.0, -5.0, -0.5)
)

print("Reverse Bias Simulation Configured")
print("="*40)
print("Voltage sweep: 0V to -5V")
print("Step size: -0.5V")

In [ ]:
# Depletion width vs reverse bias
V_reverse = np.linspace(0, 5, 50)

W_vs_V = np.sqrt(2 * eps_si / q * (1/Na + 1/Nd) * (Vbi + V_reverse)) * 1e4  # um

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(V_reverse, W_vs_V, 'b-', linewidth=2)
plt.xlabel('Reverse Bias (V)', fontsize=12)
plt.ylabel('Depletion Width (um)', fontsize=12)
plt.title('Depletion Width vs Reverse Bias', fontsize=14)
plt.grid(True, alpha=0.3)

# Capacitance vs reverse bias
A = 1e-8  # Device area (cm^2)
C = eps_si * A / (W_vs_V * 1e-4)  # F

plt.subplot(1, 2, 2)
plt.plot(V_reverse, C * 1e15, 'r-', linewidth=2)
plt.xlabel('Reverse Bias (V)', fontsize=12)
plt.ylabel('Capacitance (fF)', fontsize=12)
plt.title('Junction Capacitance vs Reverse Bias', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 6. Effect of Doping Concentration

Let's explore how doping affects the diode characteristics.

In [ ]:
# Create diodes with different doping levels
doping_levels = [1e15, 1e16, 1e17, 1e18]

print("Creating diodes with different doping levels:")
print("="*50)

for doping in doping_levels:
    sim = create_pn_diode(
        length=2.0,
        p_doping=doping,
        n_doping=doping,
        log_iv=True,
        iv_file=f"iv_{doping:.0e}",
        forward_sweep=(0.0, 0.8, 0.05)
    )
    
    # Calculate theoretical Vbi
    Vbi_calc = kT * np.log(doping * doping / ni**2)
    W_calc = np.sqrt(2 * eps_si / q * (2/doping) * Vbi_calc) * 1e4
    
    print(f"Na = Nd = {doping:.0e} cm^-3:")
    print(f"  Vbi = {Vbi_calc:.3f} V")
    print(f"  W = {W_calc:.4f} um")

In [ ]:
# Visualize the effect of doping
plt.figure(figsize=(12, 5))

# Built-in potential vs doping
doping_range = np.logspace(14, 19, 100)
Vbi_range = kT * np.log(doping_range**2 / ni**2)

plt.subplot(1, 2, 1)
plt.semilogx(doping_range, Vbi_range, 'b-', linewidth=2)
plt.xlabel('Doping Concentration (cm$^{-3}$)', fontsize=12)
plt.ylabel('Built-in Potential (V)', fontsize=12)
plt.title('Vbi vs Doping (symmetric junction)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.2)

# Depletion width vs doping (at equilibrium)
W_range = np.sqrt(2 * eps_si / q * (2/doping_range) * Vbi_range) * 1e4

plt.subplot(1, 2, 2)
plt.loglog(doping_range, W_range, 'r-', linewidth=2)
plt.xlabel('Doping Concentration (cm$^{-3}$)', fontsize=12)
plt.ylabel('Depletion Width (um)', fontsize=12)
plt.title('Depletion Width vs Doping', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 7. Complete Simulation Example

Here's a complete example combining forward and reverse characteristics:

In [ ]:
# Complete PN diode characterization
sim_complete = create_pn_diode(
    # Device geometry
    length=2.0,
    width=1.0,
    junction_position=0.5,
    
    # High-resolution mesh
    nx=300,
    ny=3,
    
    # Doping
    p_doping=1e17,
    n_doping=1e17,
    
    # Physical models
    temperature=300,
    srh=True,
    conmob=True,
    fldmob=True,
    
    # Material parameters
    taun0=1e-6,  # Electron lifetime
    taup0=1e-6,  # Hole lifetime
    
    # Output logging
    log_iv=True,
    iv_file="complete_iv",
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Voltage sweeps
    forward_sweep=(0.0, 0.8, 0.02),   # Fine forward sweep
    reverse_sweep=(0.0, -5.0, -0.25)  # Reverse sweep
)

print("Complete PN Diode Simulation")
print("="*50)
print("\nGenerated PADRE deck:")
print("-"*50)
deck = sim_complete.generate_deck()
print(deck)

---

## 8. Exercises

### Exercise 1: Asymmetric Junction
Create a diode with different doping on each side (Na = 1e16, Nd = 1e18). How does this affect:
- The built-in potential?
- The depletion region distribution?

### Exercise 2: Temperature Dependence
Simulate the diode at different temperatures (250K, 300K, 350K, 400K). Observe how:
- The I-V curve changes
- The reverse saturation current increases

### Exercise 3: Recombination Effects
Compare simulations with and without SRH recombination. How does this affect the ideality factor?

In [ ]:
# Exercise 1: Asymmetric junction
sim_asymmetric = create_pn_diode(
    p_doping=1e16,   # Lightly doped P-side
    n_doping=1e18,   # Heavily doped N-side
    log_bands_eq=True,
    forward_sweep=(0.0, 0.8, 0.05)
)

# Calculate where depletion extends
Na_asym = 1e16
Nd_asym = 1e18
Vbi_asym = kT * np.log(Na_asym * Nd_asym / ni**2)

# Depletion widths on each side
xp = np.sqrt(2 * eps_si * Vbi_asym / q * Nd_asym / (Na_asym * (Na_asym + Nd_asym))) * 1e4
xn = np.sqrt(2 * eps_si * Vbi_asym / q * Na_asym / (Nd_asym * (Na_asym + Nd_asym))) * 1e4

print("Asymmetric Junction Analysis")
print("="*40)
print(f"Na = {Na_asym:.0e} cm^-3")
print(f"Nd = {Nd_asym:.0e} cm^-3")
print(f"Vbi = {Vbi_asym:.3f} V")
print(f"Depletion in P-region: {xp:.4f} um")
print(f"Depletion in N-region: {xn:.4f} um")
print(f"Total depletion width: {xp + xn:.4f} um")
print(f"\nNote: Most depletion is in the lightly-doped P-region")

---

## Summary

In this notebook, you learned:

1. **PN Junction Physics**: Built-in potential, depletion region, I-V characteristics
2. **Creating Simulations**: Using `create_pn_diode()` with various parameters
3. **Band Diagrams**: Understanding equilibrium and biased band structures
4. **I-V Characteristics**: Forward bias exponential behavior, reverse saturation
5. **Parameter Effects**: How doping concentration affects device behavior

**Key Equations:**
- Built-in potential: $V_{bi} = \frac{kT}{q}\ln\left(\frac{N_aN_d}{n_i^2}\right)$
- Ideal diode: $I = I_s(e^{qV/kT} - 1)$

**Next**: [03 - Schottky Diode](03_Schottky_Diode.ipynb) - Metal-semiconductor junctions